In [1]:
from mxnet import nd
from mxnet.gluon import nn

net = nn.Sequential()
with net.name_scope():
    net.add(nn.Dense(256, activation = 'relu'))
    net.add(nn.Dense(10))

print(net)

Sequential(
  (0): Dense(256, Activation(relu))
  (1): Dense(10, linear)
)


/home/dyjng/anaconda3/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


In [2]:
# define MLP which is subclass of nn.Block
class MLP(nn.Block):
    # recall nn_Block.__init__ which contains prefix and params
    def __init__(self, **kwargs):
        super(MLP, self).__init__(**kwargs)
        # recall nn_Block.name_scope()
        with self.name_scope():
            self.dense0 = nn.Dense(256)
            self.dense1 = nn.Dense(10)
        
    def forward(self, x):
        return self.dense1(nd.relu(self.dense0(x)))

In [3]:
net2 = MLP()
print(net2)
net2.initialize()
x = nd.random.uniform(shape = (4, 20))
y = net2(x)
y

MLP(
  (dense0): Dense(256, linear)
  (dense1): Dense(10, linear)
)



[[ 0.05502447  0.01093244 -0.05812225 -0.00867474  0.00780752 -0.03732029
  -0.11888048 -0.01667178 -0.12706244 -0.00605519]
 [ 0.05254333 -0.03761618 -0.03303654 -0.06370584  0.02936437 -0.04790818
  -0.07402188  0.00388384 -0.09476319  0.00247342]
 [ 0.03847572 -0.01801044 -0.02936447 -0.04202728  0.00755377 -0.06616984
  -0.08015118  0.04540668 -0.08034274  0.00180145]
 [ 0.03042224 -0.04749024 -0.00121015 -0.08124933  0.03479041 -0.06163511
  -0.10677548  0.04019741 -0.1076465   0.01437488]]
<NDArray 4x10 @cpu(0)>

In [4]:
nn.Dense

mxnet.gluon.nn.basic_layers.Dense

In [5]:
print('default prefix:', net2.dense0.name)

net3 = MLP(prefix = 'another_mlp_')
print('customized prefix:', net3.dense0.name)

default prefix: mlp0_dense0
customized prefix: another_mlp_dense0


In [6]:
class Sequential(nn.Block):
    def __init__(self, **kwargs):
        super(Sequential, self).__init__(**kwargs)
    def add(self, block):
        self._children.append(block)
    def forward(self, x):
        # execute blocks one by one 
        for block in self._children:
            # recursion
            x = block(x)
        return x

In [7]:
net4 = Sequential()
with net4.name_scope():
    net4.add(nn.Dense(256, activation = 'relu'))
    net4.add(nn.Dense(10))

net4.initialize()
y = net4(x)
y


[[-0.05634359  0.09217402  0.06786803  0.00810092  0.00316704 -0.06578711
   0.02175836  0.00841999  0.0647321   0.01264806]
 [-0.0608877   0.06674264  0.08634251  0.06163288 -0.01288303 -0.01728502
  -0.00963083  0.0280523   0.02129908  0.05371749]
 [-0.04579362  0.11277001  0.0501334   0.01711009 -0.00263513 -0.04143213
   0.01833685  0.02963726  0.05529994  0.01901205]
 [-0.09248228  0.1179922   0.08974072  0.02259768  0.01704468 -0.07296751
   0.02300572  0.038479    0.05917452  0.03611853]]
<NDArray 4x10 @cpu(0)>

In [8]:
class FancyMLP(nn.Block):
    def __init__(self, **kwargs):
        super(FancyMLP, self).__init__(**kwargs)
        with self.name_scope():
            self.dense = nn.Dense(256)
            self.weight = nd.random_uniform(shape = (256, 20))
    
    def forward(self, x):
        x = nd.relu(self.dense(x))
        x = nd.relu(nd.dot(x, self.weight) + 1)
        x = nd.relu(self.dense(x))
        return x

In [9]:
fancy_mlp = FancyMLP()
print(fancy_mlp)
fancy_mlp.initialize()
y = fancy_mlp(x)
print(y.shape)

FancyMLP(
  (dense): Dense(256, linear)
)
(4, 256)


In [10]:
class RecMLP(nn.Block):
    def __init__(self, **kwargs):
        super(RecMLP, self).__init__(**kwargs)
        self.net = nn.Sequential()
        with self.name_scope():
            self.net.add(nn.Dense(256, activation = 'relu'))
            self.net.add(nn.Dense(128, activation = 'relu'))
            self.dense = nn.Dense(64)
            
        def forward(self, x):
            return nd.relu(self.dense(self.net(x)))
        
rec_mlp = nn.Sequential()
rec_mlp.add(RecMLP())
rec_mlp.add(nn.Dense(10))
print(rec_mlp)

Sequential(
  (0): RecMLP(
    (net): Sequential(
      (0): Dense(256, Activation(relu))
      (1): Dense(128, Activation(relu))
    )
    (dense): Dense(64, linear)
  )
  (1): Dense(10, linear)
)


In [11]:
class RecMLP1(nn.Block):
    def __init__(self, **kwargs):
        super(RecMLP1, self).__init__(**kwargs)
        self.net = nn.Sequential()
        with self.name_scope():
            self.dense = [nn.Dense(256), nn.Dense(128), nn.Dense(64)]
            
        def forward(self, x):
            for dense in self.dense:
                x = nn.relu(dense(self.net(x)))
            return x

rec_mlp1 = RecMLP1()
print(rec_mlp1)

RecMLP1(
  (net): Sequential(
  
  )
)
